# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 11 2023 12:28PM,254786,10,Enova-11190,Emerginnova,Released
1,Jan 11 2023 12:28PM,254786,10,Enova-11191,Emerginnova,Released
2,Jan 11 2023 12:12PM,254785,10,CTF0011588,"Citieffe, Inc.",Released
3,Jan 11 2023 12:04PM,254784,19,9018782,"Avet Pharmaceuticals Labs, Inc",Released
4,Jan 11 2023 12:03PM,254783,10,PRF-43485,Bio-PRF,Released
5,Jan 11 2023 12:03PM,254783,10,PRF-43645-BO,Bio-PRF,Released
6,Jan 11 2023 12:03PM,254783,10,PRF-43648-BO,Bio-PRF,Released
7,Jan 11 2023 12:03PM,254783,10,PRF-44056-BO,Bio-PRF,Released
8,Jan 11 2023 12:03PM,254783,10,PRF-44522,Bio-PRF,Released
9,Jan 11 2023 12:03PM,254783,10,PRF-44053-BO,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,254779,Released,2
36,254783,Released,7
37,254784,Released,1
38,254785,Released,1
39,254786,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254779,NaN,NaN,2.0
254783,NaN,NaN,7.0
254784,NaN,NaN,1.0
254785,NaN,NaN,1.0
254786,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254673,10.0,1.0,0.0
254697,0.0,0.0,1.0
254703,0.0,1.0,0.0
254727,0.0,5.0,24.0
254729,0.0,0.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254673,10,1,0
254697,0,0,1
254703,0,1,0
254727,0,5,24
254729,0,0,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254673,10,1,0
1,254697,0,0,1
2,254703,0,1,0
3,254727,0,5,24
4,254729,0,0,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254673,10,1,
1,254697,,,1
2,254703,,1,
3,254727,,5,24
4,254729,,,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 11 2023 12:28PM,254786,10,Emerginnova
2,Jan 11 2023 12:12PM,254785,10,"Citieffe, Inc."
3,Jan 11 2023 12:04PM,254784,19,"Avet Pharmaceuticals Labs, Inc"
4,Jan 11 2023 12:03PM,254783,10,Bio-PRF
11,Jan 11 2023 11:41AM,254779,10,"Citieffe, Inc."
13,Jan 11 2023 11:39AM,254777,10,MedStone Pharma LLC
17,Jan 11 2023 11:38AM,254776,10,MedStone Pharma LLC
24,Jan 11 2023 11:35AM,254774,10,Emerginnova
25,Jan 11 2023 11:35AM,254773,10,"Methapharm, Inc."
27,Jan 11 2023 11:33AM,254772,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 11 2023 12:28PM,254786,10,Emerginnova,,,2
1,Jan 11 2023 12:12PM,254785,10,"Citieffe, Inc.",,,1
2,Jan 11 2023 12:04PM,254784,19,"Avet Pharmaceuticals Labs, Inc",,,1
3,Jan 11 2023 12:03PM,254783,10,Bio-PRF,,,7
4,Jan 11 2023 11:41AM,254779,10,"Citieffe, Inc.",,,2
5,Jan 11 2023 11:39AM,254777,10,MedStone Pharma LLC,,,4
6,Jan 11 2023 11:38AM,254776,10,MedStone Pharma LLC,,,7
7,Jan 11 2023 11:35AM,254774,10,Emerginnova,,,1
8,Jan 11 2023 11:35AM,254773,10,"Methapharm, Inc.",,1,1
9,Jan 11 2023 11:33AM,254772,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 12:28PM,254786,10,Emerginnova,2,,
1,Jan 11 2023 12:12PM,254785,10,"Citieffe, Inc.",1,,
2,Jan 11 2023 12:04PM,254784,19,"Avet Pharmaceuticals Labs, Inc",1,,
3,Jan 11 2023 12:03PM,254783,10,Bio-PRF,7,,
4,Jan 11 2023 11:41AM,254779,10,"Citieffe, Inc.",2,,
5,Jan 11 2023 11:39AM,254777,10,MedStone Pharma LLC,4,,
6,Jan 11 2023 11:38AM,254776,10,MedStone Pharma LLC,7,,
7,Jan 11 2023 11:35AM,254774,10,Emerginnova,1,,
8,Jan 11 2023 11:35AM,254773,10,"Methapharm, Inc.",1,1,
9,Jan 11 2023 11:33AM,254772,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 12:28PM,254786,10,Emerginnova,2,,
1,Jan 11 2023 12:12PM,254785,10,"Citieffe, Inc.",1,,
2,Jan 11 2023 12:04PM,254784,19,"Avet Pharmaceuticals Labs, Inc",1,,
3,Jan 11 2023 12:03PM,254783,10,Bio-PRF,7,,
4,Jan 11 2023 11:41AM,254779,10,"Citieffe, Inc.",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 12:28PM,254786,10,Emerginnova,2.0,NaN,NaN
1,Jan 11 2023 12:12PM,254785,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Jan 11 2023 12:04PM,254784,19,"Avet Pharmaceuticals Labs, Inc",1.0,NaN,NaN
3,Jan 11 2023 12:03PM,254783,10,Bio-PRF,7.0,NaN,NaN
4,Jan 11 2023 11:41AM,254779,10,"Citieffe, Inc.",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 12:28PM,254786,10,Emerginnova,2.0,0.0,0.0
1,Jan 11 2023 12:12PM,254785,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Jan 11 2023 12:04PM,254784,19,"Avet Pharmaceuticals Labs, Inc",1.0,0.0,0.0
3,Jan 11 2023 12:03PM,254783,10,Bio-PRF,7.0,0.0,0.0
4,Jan 11 2023 11:41AM,254779,10,"Citieffe, Inc.",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5859420,91.0,22.0,12.0
16,1018936,3.0,1.0,0.0
19,1018937,39.0,5.0,0.0
21,764265,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5859420,91.0,22.0,12.0
1,16,1018936,3.0,1.0,0.0
2,19,1018937,39.0,5.0,0.0
3,21,764265,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,91.0,22.0,12.0
1,16,3.0,1.0,0.0
2,19,39.0,5.0,0.0
3,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,91.0
1,16,Released,3.0
2,19,Released,39.0
3,21,Released,3.0
4,10,Executing,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,21
Status,,,,
Completed,12.0,0.0,0.0,0.0
Executing,22.0,1.0,5.0,0.0
Released,91.0,3.0,39.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,21
0,Completed,12.0,0.0,0.0,0.0
1,Executing,22.0,1.0,5.0,0.0
2,Released,91.0,3.0,39.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,21
0,Completed,12.0,0.0,0.0,0.0
1,Executing,22.0,1.0,5.0,0.0
2,Released,91.0,3.0,39.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()